# Objective
This notebook contains the data cleaning and feature engineering that we have conducted for reviewer and business data.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Reading Data

After reading the data, we fix prefixes to each df so that we can easily differentiate features with same the name

In [14]:
# Load Datasets
reviewers = pd.read_csv("../data/raw/Reviewers (Users) CSV.csv")

# Reviews dataset needed to determine correct time reference for account age feature
reviews = pd.read_csv("../data/raw/Smaller_Reviews.csv")

restaurants = pd.read_csv("../data/raw/Resturants CSV.csv")

In [15]:
# Columns to exclude prefix, for ease of joining
exclude = ['reviewID', 'reviewerID', 'restaurantID']

# Add prefix to distinctively differentiate columns

reviews.columns = [
    col if col in exclude else 'review_' + col
    for col in reviews.columns
]

reviewers.columns = [
    col if col in exclude else 'reviewer_' + col
    for col in reviewers.columns
]

restaurants.columns = [
    col if col in exclude else 'restaurant_' + col
    for col in restaurants.columns
]

## Feature Engineering for Reviewers

In [16]:
# Convert reviews and reviewer dates to datetime
reviews['review_date'] = pd.to_datetime(reviews['review_date'], errors='coerce')

reviewers['reviewer_yelpJoinDate'] = pd.to_datetime(reviewers['reviewer_yelpJoinDate'], format='%B %Y', errors='coerce')

reviewers[['reviewerID', 'reviewer_yelpJoinDate']].head()

,reviewerID,reviewer_yelpJoinDate
0,bNYesZ944s6IJVowOnB0iA,2009-10-01
1,TRKxLC3y-ZvP45e5iilMtw,2011-06-01
2,0EMm8umAqXZzyhxNpL4M9g,2008-09-01
3,DlwexC7z88ymAzu45skODw,2007-10-01
4,kW2dk1CWihmh3g7k9N2G8A,2007-07-01


In [17]:
# Split reviewer location into city, state
loc_split = reviewers['reviewer_location'].str.split(',', n=1, expand=True)
reviewers['reviewer_city'] = loc_split[0].str.strip()
reviewers['reviewer_state'] = loc_split[1].str.strip()

In [18]:
df = reviews.merge(
    reviewers,
    on="reviewerID",
    how="left"
)

df = df.merge(
    restaurants,
    on="restaurantID",
    how="left"
)

In [19]:
# Convert review date
df['review_date'] = pd.to_datetime(df['review_date'])

# Account age at time of review
df['account_age_days'] = (df['review_date'] - df['reviewer_yelpJoinDate']).dt.days
df['account_age_months'] = df['account_age_days'] / 30

In [20]:
safe_review_count = df['reviewer_reviewCount'].replace(0, np.nan)

df['reviews_per_month'] = df['reviewer_reviewCount'] / df['account_age_months']

df['useful_per_review'] = df['reviewer_usefulCount'] / safe_review_count
df['cool_per_review'] = df['reviewer_coolCount'] / safe_review_count
df['funny_per_review'] = df['reviewer_funnyCount'] / safe_review_count
df['compliments_per_review'] = df['reviewer_complimentCount'] / safe_review_count
df['tips_per_review'] = df['reviewer_tipCount'] / safe_review_count
df['fans_per_review'] = df['reviewer_fanCount'] / safe_review_count
df['first_review_ratio'] = df['reviewer_firstCount'] / safe_review_count

# Engagement score
df['engagement_ratio'] = (
    df['reviewer_usefulCount'] +
    df['reviewer_coolCount'] +
    df['reviewer_funnyCount']
) / safe_review_count


## Feature Engineering for Businesses

In [21]:
# Split by "-" first
parts = df['restaurant_location'].fillna("").str.split("-")

# Extract city/state (last element)
def get_city_state(x):
    if len(x) >= 1:
        last_part = x[-1].strip()
        if ", " in last_part:
            return last_part.split(", ")
    return (None, None)

df['restaurant_city'], df['restaurant_state'] = zip(*parts.apply(get_city_state))

# Extract neighbourhood (second-to-last element)
def get_neighbourhood(x):
    if len(x) >= 2:
        return x[-2].strip()
    return None

df['restaurant_neighbourhood'] = parts.apply(get_neighbourhood)

In [22]:
# Business Profile Completeness, shady companies could have alot of missing fields. 
business_cols = [
    "restaurant_GoodforKids", "restaurant_AcceptsCreditCards", "restaurant_Parking", "restaurant_Attire", "restaurant_GoodforGroups",
    "restaurant_PriceRange", "restaurant_TakesReservations", "restaurant_Delivery", "restaurant_Takeout", "restaurant_WaiterService",
    "restaurant_OutdoorSeating", "restaurant_WiFi", "restaurant_GoodFor", "restaurant_Alcohol", "restaurant_NoiseLevel", "restaurant_Ambience",
    "restaurant_HasTV", "restaurant_Caters", "restaurant_WheelchairAccessible", "restaurant_webSite", "restaurant_phoneNumber"
]

df["restaurants_num_missing_fields"] = df[business_cols].isna().sum(axis=1)

In [23]:
# Businesses that have more features typically have higher operational bandwidth, and attract more organic reviews.
restaurant_feature_cols = [
    "restaurant_GoodforKids", "restaurant_AcceptsCreditCards", "restaurant_Parking",
    "restaurant_Attire", "restaurant_GoodforGroups", "restaurant_PriceRange",
    "restaurant_TakesReservations", "restaurant_Delivery", "restaurant_Takeout",
    "restaurant_WaiterService", "restaurant_OutdoorSeating", "restaurant_WiFi",
    "restaurant_Alcohol", "restaurant_NoiseLevel", "restaurant_Ambience",
    "restaurant_HasTV", "restaurant_Caters", "restaurant_WheelchairAccessible"
]

df['restaurant_feature_count'] = df[restaurant_feature_cols].notna().sum(axis=1)

In [24]:
# Deal with missing values
numeric_cols = [
    'restaurant_reviewCount',
    'restaurant_rating',
    'restaurant_filReviewCount'
]

df[numeric_cols] = df[numeric_cols].fillna(0)

# Highly filtered volume signals suspicious behaviour.
df["restaurant_filtered_ratio"] = df["restaurant_filReviewCount"] / (df["restaurant_reviewCount"] + 1)
df["restaurant_filtered_diff"] = df["restaurant_reviewCount"] - df["restaurant_filReviewCount"]

### Remove unneccessary fields that add no information

Furthermore, adding some of these features in might needlessly add on to the sparsity of the matrix. Hence we only keep the features that makes the most sense. 

In [25]:
cols_to_drop = [
    "restaurant_name","restaurant_address","restaurant_phoneNumber", "restaurant_location", "restaurant_Hours", "restaurant_categories",
    "restaurant_GoodforKids", "restaurant_AcceptsCreditCards", "restaurant_Parking", "restaurant_Attire", "restaurant_GoodforGroups",
    "restaurant_PriceRange", "restaurant_TakesReservations", "restaurant_Delivery", "restaurant_Takeout", "restaurant_WaiterService",
    "restaurant_OutdoorSeating", "restaurant_WiFi", "restaurant_GoodFor", "restaurant_Alcohol", "restaurant_NoiseLevel", "restaurant_Ambience",
    "restaurant_HasTV", "restaurant_Caters", "restaurant_WheelchairAccessible", "restaurant_webSite", "restaurant_phoneNumber",
    "restaurant_categories"
]

df = df.drop(columns=cols_to_drop, errors="ignore")

### Perform One Hot Encoding for model training

We do OHE for location based features because fraud reviews could be more rampant in certain locations.

In [26]:
restaurant_cols = [col for col in df.columns if col.startswith('restaurant_')]

object_cols = df[restaurant_cols].select_dtypes(include='object').columns.tolist()

for col in object_cols:
    # Get top 5 most frequent categories
    top5 = df[col].value_counts().nlargest(5).index

    # Collapse others
    df[col + '_top5'] = df[col].where(df[col].isin(top5), 'Others')

top5_cols = [col + '_top5' for col in object_cols]
df = pd.get_dummies(df, columns=top5_cols, dummy_na=False)

# Drop the original object columns
# df.drop(columns=object_cols, inplace=True)

In [27]:
new_restaurant_cols = [col for col in df.columns if col.startswith('restaurant_')]

df[new_restaurant_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   restaurant_reviewCount                         50000 non-null  float64
 1   restaurant_rating                              50000 non-null  float64
 2   restaurant_filReviewCount                      50000 non-null  float64
 3   restaurant_city                                49340 non-null  object 
 4   restaurant_state                               49340 non-null  object 
 5   restaurant_neighbourhood                       49601 non-null  object 
 6   restaurant_feature_count                       50000 non-null  int64  
 7   restaurant_filtered_ratio                      50000 non-null  float64
 8   restaurant_filtered_diff                       50000 non-null  float64
 9   restaurant_city_top5_Chicago                   500

# Reviewers & Restaurants Feature Engineering

In [28]:
# Same state match
df['same_state'] = (df['reviewer_state'] == df['restaurant_state']).astype(int)

# City Match
df['same_city'] = (df['reviewer_city'] == df['restaurant_city']).astype(int)

# Reviewer–Restaurant Location Distance (Textual)
df['location_exact_match'] = (df['same_city'] & df['same_state']).astype(int)

In [29]:
df.head(3)

,review_Unnamed: 0,review_date,reviewID,reviewerID,review_reviewContent,review_rating,review_usefulCount,review_coolCount,review_funnyCount,review_flagged,...,restaurant_state_top5_TX,restaurant_neighbourhood_top5_Lakeview,restaurant_neighbourhood_top5_Lincoln Park,restaurant_neighbourhood_top5_Near North Side,restaurant_neighbourhood_top5_Others,restaurant_neighbourhood_top5_The Loop,restaurant_neighbourhood_top5_Wicker Park,same_state,same_city,location_exact_match
0,0,2008-11-14,NILB9aTWEwVmrw7MvHrhDw,WI9UJah3bVB258wzEIh34A,"Good food, cute ambiance, good drinks, a cool ...",4,1,4,3,0,...,False,False,False,False,True,False,False,0,0,0
1,1,2011-02-02,eedzlwLfJsVhd59JWtZK5A,hSFZC_AG7OgVfeFTt6RBkQ,So we went shopping in the Design District and...,3,1,0,0,0,...,False,False,False,False,True,False,False,1,1,1
2,2,2007-03-08,y8_FAOVmQP0F0vbhgxkWxg,S3Dnc8F00c4hk8lepYWz5Q,I liked it very much ! great food and service.,5,0,0,0,1,...,False,False,True,False,False,False,False,0,0,0


### Remove unwanted features

In [30]:
# Remove columns restaurant_state, restaurant_neighbourhood, restaurant_city since we performed OHE
df.drop(columns=object_cols, inplace=True)

In [31]:
new_restaurant_cols = [col for col in df.columns if col.startswith('restaurant_')]

df[new_restaurant_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   restaurant_reviewCount                         50000 non-null  float64
 1   restaurant_rating                              50000 non-null  float64
 2   restaurant_filReviewCount                      50000 non-null  float64
 3   restaurant_feature_count                       50000 non-null  int64  
 4   restaurant_filtered_ratio                      50000 non-null  float64
 5   restaurant_filtered_diff                       50000 non-null  float64
 6   restaurant_city_top5_Chicago                   50000 non-null  bool   
 7   restaurant_city_top5_Las Vegas                 50000 non-null  bool   
 8   restaurant_city_top5_Los Angeles               50000 non-null  bool   
 9   restaurant_city_top5_New York                  500

In [32]:
df.head(3)

,review_Unnamed: 0,review_date,reviewID,reviewerID,review_reviewContent,review_rating,review_usefulCount,review_coolCount,review_funnyCount,review_flagged,...,restaurant_state_top5_TX,restaurant_neighbourhood_top5_Lakeview,restaurant_neighbourhood_top5_Lincoln Park,restaurant_neighbourhood_top5_Near North Side,restaurant_neighbourhood_top5_Others,restaurant_neighbourhood_top5_The Loop,restaurant_neighbourhood_top5_Wicker Park,same_state,same_city,location_exact_match
0,0,2008-11-14,NILB9aTWEwVmrw7MvHrhDw,WI9UJah3bVB258wzEIh34A,"Good food, cute ambiance, good drinks, a cool ...",4,1,4,3,0,...,False,False,False,False,True,False,False,0,0,0
1,1,2011-02-02,eedzlwLfJsVhd59JWtZK5A,hSFZC_AG7OgVfeFTt6RBkQ,So we went shopping in the Design District and...,3,1,0,0,0,...,False,False,False,False,True,False,False,1,1,1
2,2,2007-03-08,y8_FAOVmQP0F0vbhgxkWxg,S3Dnc8F00c4hk8lepYWz5Q,I liked it very much ! great food and service.,5,0,0,0,1,...,False,False,True,False,False,False,False,0,0,0


# Mini EDA

In [34]:
df[df['review_flagged'] == 1]['restaurant_state_top5_IL'].value_counts()

restaurant_state_top5_IL
True    8303
Name: count, dtype: int64

# Write data 

In [22]:
df.to_csv("../data/processed/Reviewer_Business_Feature_Engineered.csv")